1. I have a  MATHS DATASET with problems  and solution.
2. I need to clean it, so that the problem is in text and mathematical notation in LaTeX form along with category like 'Algebra','Combinatorics','Number Theory' or 'Geometry'
3. EXAMPLE : (*Original problem*)Jenn randomly chooses a number J from 3 , 1 , 3,\ldots , 22 , 21 . Bela then randomly chooses a number B from 2 , 3 , 3,\ldots , 17 , 18 distinct from J . The value of B - J is at least 5 with a probability that can be expressed in the form \tfrac{m}{n } , where m and n are relatively prime positive integers . Solve m+n .
   
(*Cleaned problem*) Jenn randomly chooses a number \( J \) from the set  
\[
\{3, 1, 3, \dots, 22, 21\}
\]
Bela then randomly chooses a number \( B \) from the set  
\[
\{2, 3, 3, \dots, 17, 18\}
\]
such that \( B \neq J \). The probability that \( B - J \geq 5 \) can be expressed as  
\[
\frac{m}{n}
\]
where \( m \) and \( n \) are relatively prime positive integers. Solve for \( m+n \)

4.Using LLM to achieve it


**Install necessary Packages**

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [3]:
import torch
import pandas as pd
import re
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
from transformers import BitsAndBytesConfig
from concurrent.futures import ThreadPoolExecutor

In [4]:
# Path to LLaMA model on Kaggle 
base_model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

# BitsAndBytes Configuration for 4-bit Quantization
#Using 4-bit quantization to reduce VRAM usage (ideal for Dual T4 GPUs on Kaggle) while maintaining reasonable model accuracy.

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

# Load the LLaMA model with the provided configurations
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto", #automatically spread across available Dual T4 GPUs
    torch_dtype="float16",
    quantization_config=bnb_config,
)

# Disable caching (important when performing inference)
model.config.use_cache = False #avoids excessive memory usage
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

**Checking model response with a sample input**

In [7]:
# Load the tokenizer
#To convert text into numerical tokens
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Set the padding token to EOS token (common practice when the model doesn't have a pad token)
tokenizer.pad_token_id = tokenizer.eos_token_id

input_text = "Henry and 3 of his friends order 7 pizzas for lunch. Each pizza is cut into 8 slices. If Henry and his friends want to share the pizzas equally, how many slices can each of them have?"

# Tokenize the input text and provide the attention_mask explicitly
#truncation=True - Truncates long text to fit within the model’s max token limit.
#padding=True- Ensures all inputs have the same length (important for batching).

inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

# Ensure the input_ids are moved to the same device as the model
input_ids = inputs['input_ids'].to(model.device)
attention_mask = inputs['attention_mask'].to(model.device)  # Explicitly pass attention mask

# input_ids, attention_mask - Provides tokenized input and tells the model which tokens to focus on
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=150)

# Converts the numerical token output back into human-readable text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Henry and 3 of his friends order 7 pizzas for lunch. Each pizza is cut into 8 slices. If Henry and his friends want to share the pizzas equally, how many slices can each of them have? 
Step 1: First, let's calculate the total number of slices. Since there are 7 pizzas and each pizza is cut into 8 slices, the total number of slices is 7 * 8 = 56.
Step 2: Now, let's calculate the number of slices each person can have. There are 4 people in total (Henry and his 3 friends), so each person can have 56 / 4 = 14 slices.

The final answer is: $\boxed{14}$


In [12]:
import pandas as pd

# Load dataset
file_path = "/kaggle/input/maths-plain-text/consolidated_plain_text.csv"
df = pd.read_csv(file_path)


In [13]:
df.head()

,Unique_id,problem_statement,solution,MOD
0,GEN_AIME_AUG_Q1,"Jenn randomly chooses a number J from 3 , 1 , ...",29.0,29.0
1,GEN_AIME_AUG_Q2,"Triangle ABC has side lengths AB=4 , BC=5 , an...",32.0,32.0
2,GEN_AIME_AUG_Q3,A tennis player computes her win ratio by divi...,164.0,164.0
3,GEN_AIME_AUG_Q4,Let P(x ) be a quadratic polynomial with compl...,85.0,85.0
4,GEN_AIME_AUG_Q5,"In right triangle ABC , \angle C=90^\circ . ...",351.0,351.0


**Taking a sample of the data for debugging**

In [28]:
df_slice=df[20:21]

In [29]:
df_slice.head()

,Unique_id,problem_statement,solution,MOD
20,GEN_AIME_AUG_Q21,Let S be the Summation of all numbers of the f...,248.0,248.0


In [ ]:
# Define the base model name and configuration
base_model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",  # Automatically use available GPUs
    torch_dtype="float16",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token to eos token if needed


**Using a structured prompt for LLaMA to output LaTeX-formatted text.**

In [23]:
def clean_and_categorize(problem_text):
    
    return f"""
### TASK: Format the following math problem in proper LaTeX notation.

#### INSTRUCTIONS:
 **Do NOT solve the problem.**  
 **Convert all mathematical expressions, equations, and numbers into LaTeX syntax.**  

#### INPUT:  
{problem_text}  

#### OUTPUT (Formatted in LaTeX):  

"""

**This function sends the prompt to LLaMA 3.1, processes it, and returns the cleaned LaTeX-formatted problem**

In [25]:

def generate_cleaned_problem(problem_statement, device):
    """Generate cleaned LaTeX problem using LLaMA 3.1."""
    prompt = clean_and_categorize(problem_statement)

    # Tokenize input and send to the appropriate GPU
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad(), torch.cuda.amp.autocast():  # Optimize GPU memory usage
        outputs = model.generate(
            **inputs,
            max_length=500,
            num_beams=4,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    decoder_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("-----Processed Output----------", decoder_output)  # For debugging
    return decoder_output

**This function splits a dataset of math problems and parallelizes processing across two GPUs**

In [26]:
def process_dataset(df):
    """Distribute dataset processing across multiple GPUs."""
    #Assigns two CUDA devices (GPUs)
    device_0 = torch.device("cuda:0")
    device_1 = torch.device("cuda:1")

    # Split dataset into two halves for parallel processing
    batch_0 = df.iloc[:len(df)//2]['problem_statement'].tolist()
    batch_1 = df.iloc[len(df)//2:]['problem_statement'].tolist()

    results_0, results_1 = [], []
    
    #Uses ThreadPoolExecutor for parallel execution:
    with ThreadPoolExecutor() as executor:
        # Each batch is processed separately on different GPUs.
        #This runs generate_cleaned_problem() on each problem statement in parallel
        future_0 = executor.submit(lambda: [generate_cleaned_problem(p, device_0) for p in batch_0])
        future_1 = executor.submit(lambda: [generate_cleaned_problem(p, device_1) for p in batch_1])

        results_0 = future_0.result()
        results_1 = future_1.result()
    #Combines the results from both GPUs and returns the formatted problems.
    return results_0 + results_1

**Apply processing to Dataset**

In [30]:
# Process and add cleaned problems
df_slice["cleaned_problem"] = process_dataset(df_slice)

<ipython-input-25-8b2bf2e0dd42>:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():  # Optimize GPU memory usage
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


-----Processed Output---------- 
### TASK: Format the following math problem in proper LaTeX notation.

#### INSTRUCTIONS:
 **Do NOT solve the problem.**  
 **Convert all mathematical expressions, equations, and numbers into LaTeX syntax.**  

#### INPUT:  
Let S be the Summation of all numbers of the form \frac{a}{b }, where a and b are relatively prime positive divisors of 1002. What is the greatest integer that does not exceed \frac{S}{10 }?  

#### OUTPUT (Formatted in LaTeX):  

\begin{align*}
S &amp;= \sum_{\substack{\text{relatively prime } a,b \\ \gcd(a, b) = 1 \\ ab \, \text{|}\,  2^1 \cdot 3^2 \cdots 23^4}}\frac{\left\lfloor \sqrt{ab} \right\rfloor}{ab}\\
&amp;\qquad \times \left( \prod_{p \in \mathcal{P}} p^{\min\left(\left \lceil \log_p (ab)\right \rceil, e_p\right)} \kern-0.5ex \vphantom{\sum}\right)\\
\textbf{where } \quad
e_p = \max\{e_2,e_3,\ldots,e_{23}\},\quad e_i = v_i + 4, \\
v_i &= \min \{v \mid v\in\mathbb{Z}, v \geq 0, p_i^v\mid ab\}, \\
p_i &\equiv i \pmod{23}, 

<ipython-input-30-b72f0dca56bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice["cleaned_problem"] = process_dataset(df_slice)


In [ ]:
df_slice.head()

**LaTeX-formatted math problems using Matplotlib and displays them as images**

In [ ]:
import matplotlib.pyplot as plt

def display_latex(math_text):
    fig, ax = plt.subplots(figsize=(12, 5), dpi=150) 
    ax.text(1, 1, f"$\\displaystyle {math_text}$", 
            fontsize=20, ha='center', va='center')
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.set_frame_on(False)  
    plt.show()

# Example usage
display_latex(df_slice["cleaned_problem"]) 

**OPENAI CODE**

In [ ]:
!pip install openai==0.28

In [ ]:
from kaggle_secrets import UserSecretsClient
import openai

# Access the OpenAI API key from Kaggle Secrets
user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openAI_API_key")  # Access your stored API key securely

# Check available models to ensure GPT-4 is listed
models = openai.Model.list()
print("Available models:")
for model in models['data']:
    print(model['id'])

In [ ]:
# Example API call using GPT-4 model
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},  # System message to set the behavior of the assistant
        {"role": "user", "content": "How do I check if a Python object is an instance of a class?"}  # User query
    ]
)

# Print the response from the assistant
print(response['choices'][0]['message']['content'])

In [ ]:
# Combine all problem statements into a single string with a delimiter
problems_batch = "\n\n".join([f"Problem {i+1}: {problem}" for i, problem in enumerate(df_slice['problem_statement'])])

# Prompt to clean and categorize multiple problems
prompt = f"Clean and categorize the following math problems. Preserve the LaTeX notation where applicable. Categorize each problem into one of the following categories: Algebra, Combinatorics, Geometry, Number Theory.\n\n{problems_batch}"

# Request to OpenAI API to clean and categorize all problems in a single request
response = openai.ChatCompletion.create(
    model="gpt-4o",  # Using the gpt-4o model
    messages=[
        {"role": "system", "content": "You are a helpful assistant that cleans mathematical problem statements while preserving LaTeX notation and categorizes them."},
        {"role": "user", "content": prompt}
    ]
)

# Extract cleaned problems and categories from the response
cleaned_responses = response['choices'][0]['message']['content'].split("\n\n")  # Split by delimiter

# Process the results to fill in the DataFrame
cleaned_problems = []
categories = []
for cleaned_response in cleaned_responses:
    # Example format of cleaned_response: "Problem 1: Cleaned problem text\nCategory: Algebra"
    problem_text, category = cleaned_response.rsplit("\n", 1)
    cleaned_problems.append(problem_text.strip())
    categories.append(category.strip().replace("Category: ", ""))

# Add new columns to the DataFrame
df_slice['cleaned_problem'] = cleaned_problems
df_slice['category'] = categories

# Print the updated DataFrame
print(df_slice)

In [ ]:
df_slice["problem_statement"].tolist()

In [ ]:
df_slice["cleaned_problem"].tolist()


**USING SIMPIFY**

In [1]:
import pandas as pd
import re
from sympy import sympify, latex

**USING SYMPY - a more generalized approach ,which might not suit this dataset as each problem has a different LaTeX format error**

In [14]:
import re
from sympy import sympify, latex

def convert_to_latex(problem_text):
    try:
        problem_text = problem_text.replace("\\fra c", "\\frac")  # Fix typo if present

        # Extract math-like expressions (numbers, equations, operators)
        math_expressions = re.findall(r'(\$[^$]*\$|[\d\+\-\*/\^\(\)\=]+)', problem_text)

        for expr in math_expressions:
            if expr.startswith("$") and expr.endswith("$"):
                continue  # Already valid LaTeX format
            
            try:
                latex_expr = latex(sympify(expr)) 
                problem_text = problem_text.replace(expr, f"${latex_expr}$")
            except:
                continue  # Skip if conversion fails

        # Ensure double dollar signs are fixed
        problem_text = re.sub(r'\$\$', r'$', problem_text)  

    except Exception as e:
        print(f"Error processing: {problem_text}\nError: {e}")
        return problem_text  

    return problem_text

# Apply LaTeX conversion function
df["cleaned_problem"] = df["problem_statement"].apply(convert_to_latex)


In [15]:
df.head()

,Unique_id,problem_statement,solution,MOD,cleaned_problem
0,GEN_AIME_AUG_Q1,"Jenn randomly chooses a number J from 3 , 1 , ...",29.0,29.0,"Jenn randomly chooses a number J from $$3$$ , ..."
1,GEN_AIME_AUG_Q2,"Triangle ABC has side lengths AB=4 , BC=5 , an...",32.0,32.0,"Triangle ABC has side lengths AB=4 , BC=5 , an..."
2,GEN_AIME_AUG_Q3,A tennis player computes her win ratio by divi...,164.0,164.0,A tennis player computes her win ratio by divi...
3,GEN_AIME_AUG_Q4,Let P(x ) be a quadratic polynomial with compl...,85.0,85.0,Let P(x ) be a quadratic polynomial with compl...
4,GEN_AIME_AUG_Q5,"In right triangle ABC , \angle C=90^\circ . ...",351.0,351.0,"In right triangle ABC , \angle C=90^\circ . ..."


In [ ]:
def categorize_problem(problem_text):
    categories = {
        "Algebra": ["polynomial", "equation", "quadratic", "roots", "function", "expression", "series", "logarithm", "exponential"],
        "Geometry": ["triangle", "circle", "angle", "radius", "perpendicular", "area", "length", "volume", "sector", "bisector"],
        "Combinatorics": ["permutation", "combination", "arrangement", "ways", "choose", "counting", "subset", "probability"],
        "Number Theory": ["divisible", "prime", "gcd", "lcm", "modulo", "integer", "digits", "factor", "divisors", "remainder"]
    }
    
    for category, keywords in categories.items():
        if any(keyword in problem_text.lower() for keyword in keywords):
            return category
    
    return "Uncategorized"


df["category"] = df["problem_statement"].apply(categorize_problem)


In [ ]:
df.head()

In [ ]:
uncategorized_count = df[df["category"] == "Uncategorized"].shape[0]
print(f"Number of Uncategorized Problems: {uncategorized_count}")


In [ ]:
print(df["category"].value_counts())
